### wps_climdex_dtr

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf)'s [climdex.dtr](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=6) function

**climdex.dtr** computes the mean daily diurnal temperature range.  The frequency of observationcan be either monthly or annual.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.output_handling import rda_to_vector, auto_construct_outputs
from wps_tools.testing import get_target_url
from quail.utils import test_rda_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_dtr?

Signature:
quail.climdex_dtr(
    climdex_input=None,
    loglevel='INFO',
    output_file='output.rda',
    freq='monthly',
    output_formats=None,
)
Docstring:
Computes the mean daily diurnal temperature range.

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
freq : {'monthly', 'annual'}string
    Time frequency to aggregate to
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/code/birds/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base.py-9>
Type:      method


#### Run wps_climdex_dtr Process with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="dtr_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_dtr(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            freq="annual",
            output_file=output_file.name,
        )
dtr_url = output.get()[0]

You can also pass rds input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="dtr_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_dtr(
            climdex_input=resource_filename("tests","data/climdexInput.rds"),
            output_file=output_file.name,
        )
dtr_url_rds = output.get()[0]

Run with multiple inputs

In [8]:
climdex_inputs = [
    resource_filename("tests","data/climdexInput.rds"),
    resource_filename("tests","data/climdexInput.rda")
]
with NamedTemporaryFile(suffix=".rda", prefix="dtr_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_dtr(
            climdex_input=climdex_inputs,
            output_file=output_file.name,
        )
auto_construct_outputs(output.get())

[R object with classes: ('numeric',) mapped to:
 [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
 R object with classes: ('numeric',) mapped to:
 [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan]]

Access the output with **rda_to_vector()** or **auto_construct_outputs()** from **wps_tools.output_handling**

In [9]:
dtr = rda_to_vector(dtr_url, "dtr0_ci")
# use print() to see whole vector

In [10]:
auto_construct_outputs(output.get())

[R object with classes: ('numeric',) mapped to:
 [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
 R object with classes: ('numeric',) mapped to:
 [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan]]

#### Testing output against expected output

In [11]:
test_rda_output(
    dtr_url, "dtr0_ci", "expected_dtr.rda", "expected_dtr_annual"
    )